In [13]:
import json

#import distance data
with open(r'json/distance.json') as f:
    city_distances = json.load(f)

#import input data
with open(r'json/input.json') as f:
    city_data = json.load(f)


In [14]:
#function declarations 

#get distance function
def get_distance(start:str, end:str, data):
    dis = data[start][end] 
    if (type(data[start][end]) == str):
        return 0
    else:
        return dis
    

#get closest depot
def get_depot(city:str, data:dict):
    depots = ['Goderich', 'Toronto', 'Picton']
    lowest_dist = 1E9
    closest_depot = ""

    for depot in depots:
        dist = get_distance(city, depot, data)
        
        if (dist != 0 and dist < lowest_dist):
            lowest_dist = dist
            closest_depot = depot
            print(closest_depot)
    return closest_depot


In [15]:
#Class declaration
class Truck:
    def __init__(self, starting_depot) -> None:
        
        # Truck constants
        self.hourly_wage = 20
        self.overtime_wage = 30
        self.speed = 100
        self.cost_per_km = 0.68 + self.hourly_wage * self.speed / 100
        self.cost_per_km_overtime = 0.68 + self.overtime_wage * self.speed / 100
        
        # Truck variables
        self.home_depot = starting_depot
        self.current_city = None
        self.rest = False
        self.distance_driven_day = 0
        self.distance_driven_week = 0
        self.day_cost = 0
        self.total_cost = 0
        self.packages = 0
        # Time variables
        self.current_day = "Monday"

        # Logging
        self.deliveries_info = {
            "Monday": [],
            "Tuesday": [],
            "Wednesday": [],
            "Thursday": [],
            "Friday": [],
        }

        # To output
        self.output_dictionary = {
            "Total_cost": self.total_cost,
            "Monday": {
                
            }
        }

    def reset_day(self):
        self.distance_driven_day = 0
        self.day_cost = 0

    def generate_day_info(self, day_distance:float, day_cost:float, day_deliveries:list):
        day_info = {
                "Total_distance": day_distance,
                "Total_cost": day_cost,
        }
        i = 1
        for delivery in day_deliveries:
            day_info[f'Delivery_{i}'] = delivery
            i += 1
        
    def generate_delivery_info(self, start_t, end_t, start_city, end_city, delivery_type, start_quant, end_quant):
        delivery_info = {
            "Start_time": start_t,
            "End_time": end_t,
            "Start_destination":start_city,
            "End_destination":end_city,
            "Distance": get_distance(start_city, end_city),
            "Delivery_type":delivery_type,
            "Start_quantity":start_quant,
            "End_quantity": end_quant,
        }
        return delivery_info

    def generate_truck_output(self):
        output_dict = {
            "Total_cost": self.total_cost
        }
        weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
        for weekday in weekdays:
            output_dict[weekday] = self.generate_day_info()

    def move_truck(self, new_city:str, location_dict, city_data:dict, deliver:bool=True):
        # Calculate cost
        dist = get_distance(self.current_city, new_city, city_data)
        trip_cost = dist * self.cost_per_km

        # Update truck 
        old_city = self.current_city
        self.distance_driven_day += dist
        self.distance_driven_week += dist
        self.current_city = new_city
        
        # Deliver packages 
        start_packages = self.packages
        if deliver:
            num_packages_req = location_dict[new_city]
            if self.packages < num_packages_req:
                location_dict[new_city] -= self.packages
                self.packages = 0
        
        # Load up on packages 
        else:
            self.packages = 0
        
        # Create delivery log
        delivery_type = "Delivery"
        if not deliver:
            delivery_type = "Re-loading"

        self.deliveries_info[self.current_day] = (self.generate_delivery_info(self, 0, 0, old_city, new_city, delivery_type, start_packages, self.packages))
        return (self.generate_delivery_info(self, 0, 0, old_city, new_city, delivery_type, start_packages, self.packages))
            

    def kill_trucker(self) -> None:
        self.rest = True

In [16]:
#main algorithm
Trucks = []

for i, city in enumerate(city_data):
    closest = get_depot(city, city_distances)
    while (city_data[city] >= 0):
        new_truck = Truck(closest)
        new_truck.move_truck(city, closest, city_data)
        Trucks.append(new_truck)

Goderich


KeyError: None

In [17]:
#outputing to file
output_dictionary = json.dumps(Trucks, indent=4)

with open(r"json/output.json", "w") as outfile:
    for item in Trucks:
        outfile.write(output_dictionary)